In [ ]:
!pip install -U databento

import databento as db
import pandas as pd
import math
from tqdm import tqdm

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 80)

In [ ]:
# data_file_path = '/data/workspace_files/databento/ITCH-MBP/xnas-itch-20230508.mbp-10.dbn.zst'
data_file_path = 'xnas-itch-20230508.mbp-10.dbn.zst'
stored_data = db.DBNStore.from_file(data_file_path)

# Convert to dataframe
df = stored_data.to_df()
#print(df.head())

msft_df = df[(df.publisher_id == 2) & (df.symbol == 'MSFT')]
del msft_df['publisher_id']
del msft_df['rtype']
del msft_df['ts_in_delta']
del msft_df['instrument_id']
del msft_df['sequence']

for i in range(10):
    del msft_df[f'bid_ct_{i:02.0f}']
    del msft_df[f'ask_ct_{i:02.0f}']
    
msft_df.head()

In [ ]:
# %%time

class SingleTickerSimulatorBase:

    NO_OF_LEVELS = 10
    
    # price -> size
    BID_ALGO_ORDERS = {}
    ASK_ALGO_ORDERS = {}
    
    # Algo position
    ALGO_POSITION = 0

    # Algo fills 
    ALGO_FILLS = []
    
    # Level -> (price, size), creating after combining the market orderbook and simulator orders
    BID_SIM_ORDER_BOOK = {}
    ASK_SIM_ORDER_BOOK = {}
    
    # Maintain timestamp
    current_ts = 0
    

    def __init__(self, ticker):
        self.ticker = ticker
        
    def process_trade(self, trade_price, trade_size, trade_depth):
        
        if trade_price in self.BID_ALGO_ORDERS:
            orders_dict = self.BID_ALGO_ORDERS
            orderbook_dict = self.BID_SIM_ORDER_BOOK
            side = 1
            
        elif trade_price in self.ASK_ALGO_ORDERS:
            orders_dict = self.ASK_ALGO_ORDERS
            orderbook_dict = self.ASK_SIM_ORDER_BOOK
            side = -1
        
        else:
            return
            
        order_size = orders_dict[trade_price]
        book_size = orderbook_dict[trade_depth][1]
        
        # Assume our fill is the fraction of the book this trade cleared
        executed_size = math.ceil(order_size * min(trade_size / book_size, 1))

        self.ALGO_FILLS.append((trade_price, executed_size * side))
        self.ALGO_POSITION += executed_size * side
        
        orders_dict[trade_price] -= executed_size
        
        if orders_dict[trade_price] == 0:
            del orders_dict[trade_price]


    def process_orderbook_update(self, raw_orderbook_row):
        
        timestamp = raw_orderbook_row['ts_event']
        action = raw_orderbook_row['action']        
        
        if self.current_ts == 0:
            self.current_ts = timestamp
        
        assert timestamp >= self.current_ts, f"WTF? Trying to go back in time? {timestamp=} {self.current_ts=}"
        
        # Whenever the event type is trade, it doesn't update the orderbook in that event. Wait for the next event for the updated orderbook
        if action == 'T':
            # Create fills for the algo if price matches, else do nothing
            
            trade_price = raw_orderbook_row['price']
            trade_size = raw_orderbook_row['size']
            trade_depth = raw_orderbook_row['depth']
            
            self.process_trade(trade_price, trade_size, trade_depth)
                    
        # For now treat all other actions as same. Just construct the combined market and algo orderbook
        # TODO: Add queue position logic and compute position better using ADD and CANCEL events
        else:            
            for i in range(self.NO_OF_LEVELS):    
                bid_px = raw_orderbook_row[f'bid_px_{i:02.0f}']
                ask_px = raw_orderbook_row[f'ask_px_{i:02.0f}']
                bid_sz = raw_orderbook_row[f'bid_sz_{i:02.0f}']
                ask_sz = raw_orderbook_row[f'ask_sz_{i:02.0f}']
                
                bid_sz += self.BID_ALGO_ORDERS.get(bid_px, 0)
                ask_sz += self.ASK_ALGO_ORDERS.get(ask_px, 0)
                
                self.BID_SIM_ORDER_BOOK[i] = (bid_px, bid_sz)
                self.ASK_SIM_ORDER_BOOK[i] = (ask_px, ask_sz)
        
        self.current_ts = timestamp
   
    def run_sim(self, data_df):

        
        cnt = 0
        
        for row in data_df.iterrows():
            
            self.process_orderbook_update(row[1])
            
            if row[1]['flags'] >= 128:
                self.run_algo(self.BID_SIM_ORDER_BOOK, self.ASK_SIM_ORDER_BOOK, self.ALGO_POSITION, self.BID_ALGO_ORDERS, self.ASK_ALGO_ORDERS)
                
            cnt += 1
            
            if cnt % 20000 == 0:
                print(f"{self.current_ts=} \n{self.BID_SIM_ORDER_BOOK=} \n{self.ASK_SIM_ORDER_BOOK=} \n{self.BID_ALGO_ORDERS=} \n{self.ASK_ALGO_ORDERS=} \n{self.ALGO_POSITION=}")
                
    
    
    ## Interface for the BOT

    def place_order(self, price, size, side='ASK'):
        
        if side == 'ASK':
            orderbook_dict = self.ASK_ALGO_ORDERS
        else:
            orderbook_dict = self.BID_ALGO_ORDERS
            
        orderbook_dict[price] = size

    def cancel_order(self, price, side='ASK'):
        
        if side == 'ASK':
            orderbook_dict = self.ASK_ALGO_ORDERS
        else:
            orderbook_dict = self.BID_ALGO_ORDERS
        
        if price in orderbook_dict:
            del orderbook_dict[price]

    #To be orderwritten by the bot
    def run_algo(self, bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders):
        pass

In [ ]:
## QUOTE ALGO 

In [ ]:
msft_df.index = pd.to_datetime(msft_df.index)
msft_df.ts_event = pd.to_datetime(msft_df.ts_event)


### proportion of inform traders

PI = 0.15

### last transactions window
ROLLING = 20

###### Quote algorithm ######

''' this algorithm offers a highly simplified approach to quote bid and ask processing. 
I've made a lot of assumptions to make the quote process possible. Feel free to modify this code
Every time there is a new trade a propose a new quote
'''


def get_quotes(df):
    df_trade = df[df.action == 'T']
    ### mu_t_1 is the last update
    df_trade.loc[:,['mu_t_1']] = df_trade.loc[:,'price'].shift(1)
    df_trade.loc[:,['sigma']] = df_trade.loc[:,'price'].rolling(ROLLING).std()
    df_trade.loc[:,['monotony']] = (np.sign(df_trade.loc[:,'price'].pct_change()).replace(0)+1)/2
    df_trade.loc[:,['theta']]= df_trade.loc[:,'monotony'].rolling(ROLLING).mean()

    #Proportion of inform trader
    pi = PI

    df_trade.loc[:,['A_T']] = df_trade.mu_t_1 + df_trade.sigma*2*pi*df_trade.theta.shift(1)*(1-df_trade.theta.shift(1))/\
        (1+pi*(2*df_trade.theta.shift(1)-1))
    df_trade.loc[:,['B_T']] = df_trade.mu_t_1 - df_trade.sigma*2*pi*df_trade.theta.shift(1)*(1-df_trade.theta.shift(1))/\
         (1-pi*(2*df_trade.theta.shift(1)-1))
    
    df_quote = df.join(df_trade[['B_T','A_T']],how='left')
    df_quote.sort_index(inplace = True)
    df_quote.A_T.ffill(inplace = True)
    df_quote.B_T.ffill(inplace = True)
    return df_quote

df_trade = msft_df.reset_index()
df_trade = df_trade[df_trade.action == 'T']

df_quote = get_quotes(msft_df.reset_index())
df_quote

In [ ]:
## BOT

In [ ]:
class BOT3(SingleTickerSimulatorBase):


    def __init__(self, ticker, max_inventory, target_inventory, order_size_ratio, volatility_threshold, quotes_df=None):
        super().__init__(ticker)
        self.max_inventory = max_inventory
        self.target_inventory = target_inventory
        self.order_size_ratio = order_size_ratio
        self.volatility_threshold = volatility_threshold
        self.quotes_df = quotes_df


#     def run_sim(self, data_df):
#         if self.quotes_df is None:
#             raise ValueError("Quotes DataFrame not provided")
        
#         for index, row in data_df.iterrows():
#             quotes_row = self.quotes_df.loc[self.quotes_df['ts_event'] == row['ts_event']]
#             if not quotes_row.empty:
#                 self.current_bid_price = quotes_row['B_T'].iloc[quotes_row]
#                 self.current_ask_price = quotes_row['A_T'].iloc[quotes_row]
#                 self.process_orderbook_update(row)
                
#                 if row['flags'] >= 128:
#                     self.run_algo(self.BID_SIM_ORDER_BOOK, self.ASK_SIM_ORDER_BOOK, self.ALGO_POSITION, self.BID_ALGO_ORDERS, self.ASK_ALGO_ORDERS)


    def run_algo(self, bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders):
        self.manage_orders(bid_orderbook, ask_orderbook, bid_orders, ask_orders)
        self.adjust_orders_based_on_market(bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders)

    def manage_orders(self, bid_orderbook, ask_orderbook, bid_orders, ask_orders):
        """
        Manages existing orders, canceling those that are outside the top levels of the orderbook.
        """
        self.cancel_orders_outside_top_levels(bid_orderbook, bid_orders, 'BID')
        self.cancel_orders_outside_top_levels(ask_orderbook, ask_orders, 'ASK')

    def cancel_orders_outside_top_levels(self, orderbook, orders, side):
        """
        Cancels orders that are outside the top levels of the order book.
        """
        for price in list(orders.keys()):
            if not self.is_order_within_top_levels(price, orderbook):
                self.cancel_order(price, side=side)

    def is_order_within_top_levels(self, price, orderbook):
        """
        Checks if an order is within the top levels of the order book.
        """
        top_level_price = orderbook[0][0]
        bottom_level_price = orderbook[self.NO_OF_LEVELS - 1][0]
        return top_level_price <= price <= bottom_level_price
    
   
        
    def adjust_orders_based_on_market(self, bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders):

        spread = ask_orderbook[0][0] - bid_orderbook[0][0]
        is_volatile = spread > self.volatility_threshold
        #bid_price = self.quotes_df['B_T'][0]
        #ask_price = self.quotes_df['A_T'][0]
        
        current_ts = self.current_ts.floor('us')
        #quotes_row = self.quotes_df.loc[self.quotes_df['ts_event'] == self.current_ts.floor('us')]
        quotes_row = self.quotes_df.loc[self.quotes_df['ts_event'] == current_ts]
        
        if not quotes_row.empty:

            bid_price = quotes_row['B_T'].iloc[0]
            ask_price = quotes_row['A_T'].iloc[0]

        else: 
            bid_price = bid_orderbook[0][0] + spread / 2
            ask_price = bid_orderbook[0][0] + spread / 2
            
            bid_price_adjustment, ask_price_adjustment = self.calculate_price_adjustment(spread, inventory, is_volatile)
            bid_price = bid_price - bid_price_adjustment
            ask_price = ask_price + ask_price_adjustment


        self.place_or_adjust_order(bid_orderbook, bid_orders, inventory, 'BID', bid_price, is_volatile)
        self.place_or_adjust_order(ask_orderbook, ask_orders, inventory, 'ASK', ask_price, is_volatile)
        

    def calculate_price_adjustment(self, spread, inventory, is_volatile):
        # Adjust based on inventory and volatility
        adjustment_factor = 0.1 if is_volatile else min(0.5, abs(inventory) / self.max_inventory)
        bid_price_adjustment = spread * adjustment_factor if inventory > 0 else spread * 0.1
        ask_price_adjustment = spread * adjustment_factor if inventory < 0 else spread * 0.1
        return bid_price_adjustment, ask_price_adjustment

    def place_or_adjust_order(self, orderbook, orders, inventory, side, target_price, is_volatile):
        if len(orders) == 0 or not self.is_order_within_top_levels(target_price, orderbook):
            size = self.calculate_dynamic_order_size(orderbook, inventory, side, is_volatile)
            if size > 0:
                self.place_order(target_price, size, side=side)
            elif len(orders) > 0:
                order_price = list(orders.keys())[0]
                self.cancel_order(order_price, side=side)

    def calculate_dynamic_order_size(self, orderbook, inventory, side, is_volatile):
        level_index = self.determine_order_level(inventory, side, is_volatile)
        market_depth_size = orderbook[level_index][1]
        inventory_factor = (1 - abs(inventory) / self.max_inventory)
        return int(market_depth_size * self.order_size_ratio * inventory_factor)

    def determine_order_level(self, inventory, side, is_volatile):
        # Adjust order level based on inventory and market conditions
        if is_volatile:
            return 3 if abs(inventory) > self.max_inventory * 0.5 else 2
        else:
            return 2 if abs(inventory) > self.max_inventory * 0.5 else 1

# run the bot
bot3 = BOT3(ticker='MSFT', max_inventory=1000, target_inventory=0, order_size_ratio=0.1, volatility_threshold=0.2, quotes_df = df_quote)
bot3.run_sim(df_quote)


In [ ]:
print(bot3.ALGO_FILLS[:10])

capital = 0
position = 0
for fill in bot3.ALGO_FILLS:
    capital -= fill[0] * fill[1]
    position += fill[1]
    
print(capital, position)
pnl = capital + position * bot3.ALGO_FILLS[-1][0]
print(pnl)